In [12]:
import pandas as pd
import tensorflow as tf
import numpy as np
import nltk
import json

In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from sklearn.model_selection import train_test_split

In [14]:
df = pd.read_csv("data_kaggle(1).csv")

In [15]:
df

,no,nama,vote_average,vote_count,type,htm_weekday,htm_weekend,latitude,longitude,description,aff_link,trans_desc,types,combined,file_id,image_link
0,9,Candi Borobudur,4.7,81922,Budaya dan Sejarah,50000.0,50000.0,-7.607087,110.203623,Candi yang pernah masuk sebagai salah satu dar...,https://www.tiket.com/to-do/search?title=Candi...,"The temple, which had entered as one of the se...",Culture and history,"Culture and history The temple, which had ente...",17RA7iM5WIU1ewID8mTw47wSwzFHWxIVq,https://drive.google.com/uc?id=17RA7iM5WIU1ewI...
1,10,Candi Prambanan,4.7,71751,Budaya dan Sejarah,50000.0,50000.0,-7.751835,110.491532,Candi Prambanan adalah kompleks candi Hindu te...,https://www.tiket.com/to-do/search?title=Candi...,Prambanan Temple is the largest Hindu temple c...,Culture and history,Culture and history Prambanan Temple is the la...,1ug2o4IWvGLvxeG3HAVm5glMRvhhLgpQQ,https://drive.google.com/uc?id=1ug2o4IWvGLvxeG...
2,24,Tebing Breksi,4.4,51431,Alam,10000.0,10000.0,-7.781477,110.504576,Tebing Breksi merupakan tempat wisata yang ber...,https://www.tiket.com/to-do/search?title=Tebin...,Tebing Breksi is a tourist spot in the Sleman ...,Natural,Natural Tebing Breksi is a tourist spot in the...,1TAZvvlwY3JMuPcsQ3i9PrSxXXMKh7sVJ,https://drive.google.com/uc?id=1TAZvvlwY3JMuPc...
3,343,Gembira Loka Zoo,4.5,36337,Fauna,20000.0,25000.0,-7.806234,110.396798,Gambira Loka adalah kebun binatang yang berada...,https://www.tiket.com/to-do/search?title=Gembi...,Gambira Loka is a zoo in Yogyakarta.Contains a...,Fauna,Fauna Gambira Loka is a zoo in Yogyakarta.Cont...,1vITPWWt8GwxYguFDm3JED6DsZRDm_HsP,https://drive.google.com/uc?id=1vITPWWt8GwxYgu...
4,346,The Palace of Yogyakarta (Keraton Yogyakarta),4.6,30091,Budaya dan Sejarah,8000.0,8000.0,-7.805284,110.364203,Kompleks keraton merupakan museum yang menyimp...,https://www.tiket.com/to-do/search?title=The%2...,The palace complex is a museum that stores var...,Culture and history,Culture and history The palace complex is a mu...,1TEv0oeZu1YWhjCIoyUZSuJn4rUeofrc0,https://drive.google.com/uc?id=1TEv0oeZu1YWhjC...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,7,Bukit Pengilon,4.6,439,Alam,5000.0,5000.0,-8.185459,110.691722,Nama bukit Pengilon berasal dari bahasa Jawa y...,https://www.tiket.com/to-do/search?title=Bukit...,The name of the Bukit Pengilon comes from Java...,Natural,Natural The name of the Bukit Pengilon comes f...,1-4MUs_X2zn5KTtCr4gV8e33tHSCS3qJH,https://drive.google.com/uc?id=1-4MUs_X2zn5KTt...
122,262,Ijo Temple Wooden Mosque,4.4,33,Pantai,0.0,0.0,-7.784753,110.512844,Pasoedjoedan Al Liwung atau yang dikenal denga...,https://www.tiket.com/to-do/search?title=Ijo%2...,Pasoedjoedan Al Liwung or known as the Temple ...,Beach,Beach Pasoedjoedan Al Liwung or known as the T...,1mKPiMdkLM0yC-waHHQStdUxVNbRaXgCd,https://drive.google.com/uc?id=1mKPiMdkLM0yC-w...
123,75,Museum Batik Keraton Yogyakarta,4.9,20,Wisata Alam,20000.0,20000.0,-7.807514,110.364589,"Di Museum Batik Keraton Yogyakarta, wisatawan ...",https://www.tiket.com/to-do/search?title=Museu...,"At the Yogyakarta Palace Batik Museum, tourist...",Natural tourism,Natural tourism At the Yogyakarta Palace Batik...,1rMe-9-PsslqPJq311hgxZbLbFtePrL3h,https://drive.google.com/uc?id=1rMe-9-PsslqPJq...
124,250,Candi ASU Klaten,4.2,5,Wisata Alam,3000.0,3000.0,-7.742302,110.515587,Letaknya di Dk. Bener Ds. Bugisan Kec. Pramban...,https://www.tiket.com/to-do/search?title=Candi...,It is located on DK.Really Ds.Bugisan Kec.Pram...,Natural tourism,Natural tourism It is located on DK.Really Ds....,1Ydzo3uCHcwj4lKp_w4hcpOi3yiyle1hK,https://drive.google.com/uc?id=1Ydzo3uCHcwj4lK...


In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\N4N0C\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

df['cleaned_combined'] = df['combined'].apply(
    lambda x: ' '.join(word for word in x.split() if word.lower() not in stop_words)
)

In [18]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['cleaned_combined'])
sequences = tokenizer.texts_to_sequences(df['cleaned_combined'])
sequences = pad_sequences(sequences, padding='post')

In [19]:
label_map = {name: idx for idx, name in enumerate(set(df['types']))}
y = [label_map[name] for name in df['types']]

In [20]:
X_train, X_val, y_train, y_val = train_test_split(sequences, y, test_size=0.2, random_state=42)

In [21]:
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 128
output_dim = len(label_map)

In [22]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=sequences.shape[1],
                    input_shape=(sequences.shape[1],)))
model.add(LSTM(256, return_sequences=True, dropout=0.4, recurrent_dropout=0.2))
model.add(Bidirectional(LSTM(128, dropout=0.4, recurrent_dropout=0.2)))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(label_map), activation='softmax'))

c:\Users\N4N0C\Downloads\capstone\.conda\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
c:\Users\N4N0C\Downloads\capstone\.conda\Lib\site-packages\keras\src\layers\core\embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [23]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 182, 128)       │       287,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 182, 256)       │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 256)            │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 22)             │         1,430 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,093,590 (4.17 MB)

 Trainable params: 1,093,590 (4.17 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
optimizer = Adam(learning_rate=0.01)

In [25]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs.get('accuracy') is not None and logs.get('accuracy') > 0.90:
            print("\nReached 90% accuracy so cancelling training!")
            self.model.stop_training = True

In [26]:
X_train = np.array(X_train)
y_train = np.array(y_train)

In [27]:
X_val = np.array(X_val)
y_val = np.array(y_val)

In [28]:
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [29]:
callbacks = [myCallback()]
history = model.fit(np.array(X_train), np.array(y_train), epochs=30, batch_size=16,
                    validation_data=(np.array(X_val), np.array(y_val)), callbacks=callbacks)

Epoch 1/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 15s 818ms/step - accuracy: 0.1087 - loss: 3.0581 - val_accuracy: 0.1923 - val_loss: 2.5386
Epoch 2/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 573ms/step - accuracy: 0.4321 - loss: 2.1163 - val_accuracy: 0.3462 - val_loss: 2.1509
Epoch 3/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 467ms/step - accuracy: 0.5543 - loss: 1.8913 - val_accuracy: 0.3462 - val_loss: 1.9975
Epoch 4/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 553ms/step - accuracy: 0.4910 - loss: 1.5337 - val_accuracy: 0.3462 - val_loss: 1.7452
Epoch 5/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 462ms/step - accuracy: 0.5658 - loss: 1.6138 - val_accuracy: 0.4231 - val_loss: 1.9016
Epoch 6/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 553ms/step - accuracy: 0.5890 - loss: 1.5262 - val_accuracy: 0.3846 - val_loss: 1.8486
Epoch 7/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 515ms/step - accuracy: 0.6661 - loss: 1.3059 - val_accuracy: 0.3846 - val_loss: 1.7600
Epoch 8/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 556ms/step - accuracy: 0.6238 - loss: 1.3577 - val_accuracy: 0.3846 - val_loss

In [59]:
def recommend_places(keywords, tokenizer, model, label_map, df, top_n=5):
    # Preprocess the keywords
    keywords_sequence = tokenizer.texts_to_sequences(keywords)
    keywords_padded = pad_sequences(keywords_sequence, padding='post')

    # Predict the category
    predictions = model.predict(keywords_padded)
    predicted_category_idx = predictions.argmax(axis=1)[0]
    predicted_category = list(label_map.keys())[list(label_map.values()).index(predicted_category_idx)]

    # Filter places based on the predicted category
    filtered_places = df[df['types'] == predicted_category]

    # Get top N places
    recommended_places = filtered_places['nama'].head(top_n).tolist()

    return recommended_places

keywords = ["kraton"]
recommended_places = recommend_places(keywords, tokenizer, model, label_map, df, top_n=5)
print("Recommended Places:", recommended_places)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
Recommended Places: ['The World Landmarks - Merapi Park Yogyakarta', 'Sindu Kusuma Edupark (SKE)', 'Heha Sky View', 'XT Square', 'Embung Tambakboyo']


In [31]:
model.save('final_capstone_model.h5')

In [32]:
model.save('final_capstone_model.keras')

In [33]:
import tensorflow as tf

# Load the Keras model
model = tf.keras.models.load_model('final_capstone_model.keras')

# Create a TFLite converter
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Enable resource variables and disable lowering tensor list ops
converter.experimental_enable_resource_variables = True
converter._experimental_lower_tensor_list_ops = False  # Add this line

# Set the supported ops to include SELECT_TF_OPS
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS
]

# Convert the model
tflite_model = converter.convert()

# Save the TFLite model
with open('final_capstone_model.tflite', 'wb') as f:
    f.write(tflite_model)

c:\Users\N4N0C\Downloads\capstone\.conda\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 16 variables whereas the saved optimizer has 30 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


INFO:tensorflow:Assets written to: C:\Users\N4N0C\AppData\Local\Temp\tmpe3wf_dtu\assets


INFO:tensorflow:Assets written to: C:\Users\N4N0C\AppData\Local\Temp\tmpe3wf_dtu\assets


Saved artifact at 'C:\Users\N4N0C\AppData\Local\Temp\tmpe3wf_dtu'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 182), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 22), dtype=tf.float32, name=None)
Captures:
  2495840820240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2495840820624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2495840818512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2495840821008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2495840821776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2495928785168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2495928783440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2495928783824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2495928788048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2495928784400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2495928784016

In [34]:
model = tf.keras.models.load_model('final_capstone_model.h5')